# Json Response

In [15]:
MESSAGES = [{'role': 'user', 'content': "### Generate Google Big Query SQL query only and with no explanation\n### Here is the SQL table , with their properties:\n#\n# danish-endurance-analytics.nl2sql.amazon_orders('order_id', 'purchase_date', 'buyer_email', 'market', 'child_asin', 'e_conomic_number', 'product_marketing_category', 'product_name', 'product_pack', 'product_and_pack', 'product_category', 'product_type', 'product_size', 'product_colour', 'gross_sales', 'units_sold')\n#\n### What was the sales last Friday?\n"}, {'role': 'assistant', 'content': '```sql\nSELECT SUM(gross_sales) AS last_friday_sales\nFROM danish-endurance-analytics.nl2sql.amazon_orders\nWHERE purchase_date = DATE_SUB(CURRENT_DATE(), INTERVAL DAYOFWEEK(CURRENT_DATE()) + 1 DAY);\n```'}]

new_messages = {
    'role': 'user',
    'content': 'What was the last questiona about?'
}
MESSAGES.append(new_messages)

In [16]:
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
  model="gpt-3.5-turbo-0125",
  messages=MESSAGES,
  stream=False
)
response.choices[0].message.content

'The last question was about retrieving the sales total from last Friday from the Amazon orders table in Google BigQuery.'

In [ ]:
string_without_triple_quotes = "This is a string without triple quotes."
string_with_triple_quotes = '''{}'''.format(string_without_triple_quotes)


## Comparasion between GPT 3.5 and GPT 4 in calculating Yoy

In [3]:
content = '''Given the dataset below and statistics about this dataset, generate a SQL query in Google Big Query Syntax that answers the following: 

# User Query
Calculate the YoY Sales on a monthly level? I want to see the sales amount for each month and the percentage change from the previous year.

# Dataset - `orders.shopify`
norder_id | purchase_date | product | sales_amount

# Statistics Column: purchase_date
Total Unique Dates: 1096 Most Common Date: 2023-11-24, Frequency: 10879\nSkewness Proxy (Positive indicates skew towards later dates, Negative towards earlier dates): 0.00`
'''

In [4]:
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
  model="gpt-3.5-turbo-0125",
  messages=[
    {
      "role": "user",
      "content": content
    }
  ],
  temperature=0,
  max_tokens=500,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  n = 1
)

In [20]:
response

ChatCompletion(id='chatcmpl-95umtRAEJwbOHg3cIVI0bi4891emD', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="```sql\nWITH monthly_sales AS (\n    SELECT \n        EXTRACT(MONTH FROM purchase_date) AS month,\n        EXTRACT(YEAR FROM purchase_date) AS year,\n        SUM(sales_amount) AS total_sales\n    FROM \n        orders.shopify\n    GROUP BY \n        month, year\n)\n\nSELECT \n    CONCAT(CAST(month AS STRING), '-', CAST(year AS STRING)) AS month_year,\n    total_sales AS sales_amount,\n    ROUND((total_sales - LAG(total_sales, 12) OVER (ORDER BY year, month)) / LAG(total_sales, 12) OVER (ORDER BY year, month) * 100, 2) AS yoy_sales_percentage_change\nFROM \n    monthly_sales\nORDER BY \n    year, month\n```", role='assistant', function_call=None, tool_calls=None))], created=1711196959, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_3bc1b5746c', usage=CompletionUsage(completion_tokens=159, prompt_tok

In [6]:
 # loop through the response.choices ans save into a dataframe called df with 1 - reponse, 2 -reponse ..
import pandas as pd
df = pd.DataFrame(columns=['response'])
for i in range(10):
    df.loc[i] = response.choices[i].message.content

IndexError: list index out of range

In [83]:
df

,response
0,```sql\nWITH monthly_sales AS (\n SELECT \n...
1,```sql\nWITH monthly_sales AS (\n SELECT \n...
2,```sql\nWITH monthly_sales AS (\n SELECT \n...
3,```sql\nWITH monthly_sales AS (\n SELECT \n...
4,```sql\nWITH monthly_sales AS (\n SELECT \n...
5,```sql\nWITH monthly_sales AS (\n SELECT \n...
6,```sql\nWITH monthly_sales AS (\n SELECT \n...
7,```sql\nWITH monthly_sales AS (\n SELECT \n...
8,```sql\nWITH monthly_sales AS (\n SELECT \n...
9,```sql\nWITH monthly_sales AS (\n SELECT \n...


In [79]:
import openpyxl
# export the dataframe to an excel file called response.xlsx
df.to_excel('response_gpt3.5-4.xlsx', index = False, engine='openpyxl')

In [84]:
# count the number of columns in which the word "LAG" appears in the response
count = 0
for i in range(10):
    if "LAG" in response.choices[i].message.content:
        count += 1

In [85]:
count

10

## Self-debugging